### Преамбула к ДЗ-1

В рамках даной работы вам нужно будет поработать с hadoop, а именно:
- Сложить в хранилище hdfs файлы для выполнения задания
- Использовать его команды на практике
- Написать скрипты для MapReduce задачи и добиться их успешного выполнения

**Важно!**
Старайтесь выполнить все задания в 1 ноутбуке. От всех запущенных комманд и скриптов нужны логи успешного выполнения этих команд. Так что, пожалуйста, не очищаейте логи ячеек перед отправкой.

Также структура и порядок ячеек в ноутбуке должен быть выстроен таким образом, чтобы мы сделали **Ran All** по вашему ноутбуку, и все ячейки были выполнены (пути до вмонтированных папок и нужных файлов в них мы учтем сами)

### Заметки по работе с докером

- Если у вас ОС Windows, то для запуска сборки вполне поможет [docker desktop](https://www.docker.com/products/docker-desktop/) (должен открываться без впн), в связке с [git bash](https://gitforwindows.org/) вам станут доступны команды `docker compose` прямо из консоли гит баша, где вы сможете запустить контейнер на своем железе
- Запускать юпитер ноутбук из неймноды (свободный порт 8888 будет прокинут в конфиге, через который вы сможете обращаться к запущенному ноутбуку), его можно запустить через команду на неймноде `python3 -m notebook --ip 0.0.0.0 --port 8888 --allow_remote_access=true --no-browser --allow-root --NotebookApp.token=my_secret`, тогда вместо рандомного токена при заходе в ноутбук вам просто можно будет ввести `my_secret`
- В текущем компоуз файле будет указан шаблон вмонтированных (volume) директорий для винды, вам же стоит поменять их на пути вашей ОС. Нужные файлы для дз можете добавлять в вмонтированную папку для неймноды, чтоб их было видно успешно внутри докера, и дальше вы могли их закинуть в hdfs.
- Также в текущей ситуации доступности докер хаба мы проверили зеркало гугла, образ для этой сборки будет тянуться оттуда (указан в dockerfile)

**1. Смотрим на Hadoop Distributed File System**

В рамках этой части вам нужно будет обращаться к HDFS с помощью CLI, разместить файлы для следующих заданий в распределеннй файловой системе и выполнить несколько преобразований над ними.

Для работы файлы можно скачать по следующим ссылкам:
- Текст Шекспира на английском [ссылка](https://drive.google.com/file/d/19-ohhp5APaIgq5tjeyL9kC1vh0ur1Up9/view?usp=sharing)
- Логи посещения сайтов юзерами за некоторый промежуток времени [ссылка](https://drive.google.com/file/d/1WXyq5WVSWwJYXPuH4kyAJ5mrR3XgfO_H/view?usp=sharing)
- Данные по доходам больших компаний, их филиалов в разных странах [ссылка](https://drive.google.com/file/d/1CuUdGdrayqqO7OBYCRXwcJv64g6EhIzJ/view?usp=sharing)

Разместите их в нашем внутреннем файловом хранилище докер-контейнера с помощью HDFS CLI, для дальнейшего удобства под каждый файл стоит создать каталог с простым и понятным именем, разместить сами файлы в разных каталогах.

Набор комманд, которые вам могут в этом помочь, доступны [здесь](https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html)

В ячейках ниже должен быть полный набор комманд ваших обращей к консоли

In [7]:
import os

In [21]:
!ls

HW 1. Bash. Hadoop. MapReduce.ipynb  data.txt  data_url.csv  income_country.csv


In [23]:
# Определим текущий рабочий каталог
current_directory = os.getcwd()
print(f"Текущий рабочий каталог: {current_directory}")

Текущий рабочий каталог: /opt/hadoop/HW1


In [20]:
## вы можете обращаться к консоли из ноутбука таким способом
!hdfs dfs -ls /

Found 1 items
drwxr-xr-x   - root supergroup          0 2024-06-16 13:15 /input


In [12]:
%%bash
## или же использовать для этого меджик строчку в ячейке %%bash, как вам будет удобнее

hdfs dfs -ls /

Found 1 items
drwxr-xr-x   - root supergroup          0 2024-06-16 13:15 /input


In [32]:
!hdfs dfs -ls -R -h /input

drwxr-xr-x   - root supergroup          0 2024-06-16 13:27 /input/company_revenue
-rw-r--r--   1 root supergroup     46.4 M 2024-06-16 13:27 /input/company_revenue/income_country.csv
drwxr-xr-x   - root supergroup          0 2024-06-16 13:27 /input/shakespeare
-rw-r--r--   1 root supergroup    144.8 K 2024-06-16 13:27 /input/shakespeare/data.txt
drwxr-xr-x   - root supergroup          0 2024-06-16 13:29 /input/website_logs
-rw-r--r--   1 root supergroup     34.8 M 2024-06-16 13:29 /input/website_logs/data_url.csv


In [28]:
## ваше решение здесь
# пути к директории и файлам
local_directory = "/opt/hadoop/HW1"  
hdfs_base_directory = "/input/"

# Названия файлов и соответствующие директории в HDFS
files_to_upload = {
    "data.txt": "shakespeare",
    "data_url.csv": "website_logs",
    "income_country.csv": "company_revenue"
}

# Создание директорий в HDFS и загрузка файлов
for filename, directory in files_to_upload.items():
    local_file_path = os.path.join(local_directory, filename)
    hdfs_directory_path = os.path.join(hdfs_base_directory, directory)
    !hdfs dfs -mkdir -p {hdfs_directory_path}
    !hdfs dfs -put {local_file_path} {hdfs_directory_path}

# Проверка наличия файлов в HDFS
!hdfs dfs -ls -R -h {hdfs_base_directory}


put: `/input/shakespeare/data.txt': File exists
put: `/input/company_revenue/income_country.csv': File exists
drwxr-xr-x   - root supergroup          0 2024-06-16 13:27 /input/company_revenue
-rw-r--r--   1 root supergroup     46.4 M 2024-06-16 13:27 /input/company_revenue/income_country.csv
drwxr-xr-x   - root supergroup          0 2024-06-16 13:27 /input/shakespeare
-rw-r--r--   1 root supergroup    144.8 K 2024-06-16 13:27 /input/shakespeare/data.txt
drwxr-xr-x   - root supergroup          0 2024-06-16 13:29 /input/website_logs
-rw-r--r--   1 root supergroup     34.8 M 2024-06-16 13:29 /input/website_logs/data_url.csv


In [29]:
# Проверка содержимого каталога /input/shakespeare
!hdfs dfs -ls /input/shakespeare

Found 1 items
-rw-r--r--   1 root supergroup     148226 2024-06-16 13:27 /input/shakespeare/data.txt


In [30]:
# Проверка содержимого каталога /input/website_logs
!hdfs dfs -ls /input/website_logs

Found 1 items
-rw-r--r--   1 root supergroup   36443383 2024-06-16 13:29 /input/website_logs/data_url.csv


In [31]:
# Проверка содержимого каталога /input/company_revenue
!hdfs dfs -ls /input/company_revenue

Found 1 items
-rw-r--r--   1 root supergroup   48702760 2024-06-16 13:27 /input/company_revenue/income_country.csv


**2. Решаем задачи MapReduce**

**2.1 Подсчет слов в тексте**

В рамках данного задания вам нужно подсчитать кол-во слов в тексте Шекспира (файл приложен к ДЗ в архиве datasets), то есть необходимо реализовать базовый функционал утилиты word count.

**Важно** - подсчитывайте число только тех слов, длина которых больше 4 символов. Проводить процесс удаления знаков препинания и прочих символов **не нужно**

Ниже вам представлены ячейки, в которых вы должны описать структуру маппера/редьсюера и ниже вызвать их в bash-скрипте запуска MR-таски

In [44]:
%%writefile mapper.py
## сюда вы пишете питоновский скрипт для работы маппера

import sys

for line in sys.stdin:
    line = line.strip()
    words = line.split()
    for word in words:
        if len(word) > 4:
            print(word + "\t1")

Overwriting mapper.py


In [45]:
%%writefile reducer.py
## сюда вы пишете питоновский скрипт для работы редьюсера
import sys

current_word = None
current_count = 0

for line in sys.stdin:
    line = line.strip()
    word, count = line.split("\t", 1)
    count = int(count)
    
    if current_word == word:
        current_count += count
    else:
        if current_word:
            print(current_word + "\t" + str(current_count))
        current_word = word
        current_count = count

if current_word == word:
    print(current_word + "\t" + str(current_count))

Overwriting reducer.py


В качестве проверки ваших python-скриптов до запуска MR таски можно произвести их запуск через консольные команды 

Тогда наша задача не будет выполняться через датаноды, а только на локальной машине, но в случае ошибок в скриптах вы увидите их и сможете исправить

In [43]:
!hdfs dfs -ls /input/shakespeare/data.txt


-rw-r--r--   1 root supergroup     148226 2024-06-16 13:27 /input/shakespeare/data.txt


In [50]:
!hdfs dfs -cat /input/shakespeare/data.txt | python3 mapper.py | sort -k1,1 | python3 reducer.py


&amp;c.	1
&amp;c.]	2
'Ay.'	2
'Ay:'	2
'Heart's	4
'Hold,	1
'I'll	1
'Juliet.']	1
'Romeo	2
'Signior	1
'Then	1
'Twixt	1
'Tybalt	1
'When	1
'Where	1
'Wilt	1
'Yea,'	2
'Your	1
'banished,'	2
'dost	1
'fall'st	1
'gainst	1
'haviour	1
'music	2
'pothecary,	1
'silver	2
'tis;	1
'twas	3
'twere	1
'twill	1
'twixt	1
'twould	1
'villain'	1
'ware,	1
-What,	1
Abate	1
Above	1
Abraham	1
Abraham,	1
Abraham.	5
Accurs'd,	1
Accursed	1
Acquaint	1
Adding	1
Adieu,	1
Adversity's	1
Affection	1
Affliction	1
Afore	1
After	1
Against	1
Alack	2
Alack,	4
Alas,	4
Alike	1
Alive	1
Alone,	1
Amen!	1
Amen,	1
Among	2
Ancient	1
Angelica;	1
Anon,	2
Anon.	1
Anon?	1
Anselmo	1
Answer	1
Antony!	1
Apothecary.	5
Apothecary.]	1
Appear	1
April	1
Arise,	1
Arise;	1
Arms,	1
Ascend	1
Athwart	1
Attendants.	1
Attendants.]	2
Attendants;	1
Aurora's	1
Away,	1
Back,	1
Balthasar	1
Balthasar,	1
Balthasar.	12
Balthasar.]	4
Balthasar?	1
Banish'd	1
Bearing	1
Beautiful	1
Beauty	1
Because	3
Begot	1
Beguil'd,	1
Being	13
Believe	1
Belonging	1
Benedicite!	1
Benvo

Как только в данной проверке вы получите успешный и корректный результат, можете запустить Map Reduce в ячейке ниже

In [48]:
%%bash
## шаблон для запуска MR таски

# обязательная чистка директории, куда будем складывать результат отрабоки mr
hdfs dfs -rm -r /word_count_task || true

# запус mr таски с указанием пути до нужного jar
hadoop jar /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -D mapreduce.job.name="word-count" \
    -files ./mapper.py,./reducer.py \
    -mapper "python3 mapper.py" \
    -reducer "python3 reducer.py" \
    -input /input/shakespeare/data.txt \
    -output /word_count_task

packageJobJar: [/tmp/hadoop-unjar7655803133841050154/] [] /tmp/streamjob4703793956338461368.jar tmpDir=null
2024-06-16 13:45:23 INFO  DefaultNoHARMFailoverProxyProvider:64 - Connecting to ResourceManager at resourcemanager/172.22.0.5:8032
2024-06-16 13:45:23 INFO  DefaultNoHARMFailoverProxyProvider:64 - Connecting to ResourceManager at resourcemanager/172.22.0.5:8032
2024-06-16 13:45:23 INFO  JobResourceUploader:907 - Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1718538847154_0001
2024-06-16 13:45:24 INFO  FileInputFormat:266 - Total input files to process : 1
2024-06-16 13:45:24 INFO  JobSubmitter:202 - number of splits:2
2024-06-16 13:45:24 INFO  JobSubmitter:298 - Submitting tokens for job: job_1718538847154_0001
2024-06-16 13:45:24 INFO  JobSubmitter:299 - Executing with tokens: []
2024-06-16 13:45:24 INFO  Configuration:2854 - resource-types.xml not found
2024-06-16 13:45:24 INFO  ResourceUtils:476 - Unable to find 'resource-types.xml'.
2024-06-16 

rm: `/word_count_task': No such file or directory


Мониторить процесс работы таски можно на nodemanager по порту 8088 (уже прокинут в конфиге), там будет UI, в котором будет видно вашу запущенную задачу и её статус.

Результат работы скрипта должен выглядеть следующим образом:

```bash
word count
abtr 6852
btoad 4237
stress 1932
zen 1885
```

In [51]:
%%bash
## запустите эту команду, чтобы вывести счетчик определенных слов, которые мы указали на grep
## Это нам будет необходимо для визуального анализа результата работы вашего скрипта
## в sort можете указать тот разделитель колонок, с которым у вас результат выплевывает редьюсер

hdfs dfs -cat /word_count_task/* | grep  -E 'Romeo\.|Juliet\.|Capulet\.' | sort -t$'\t' -k2.2nr  | head -n 3

Romeo.	171
Juliet.	123
Capulet.	102


**2.2 Поиск максимального дохода у филиалов компаний по разным странам**

В данном задании нужно поработать с  информацией о доходах больших компаний и их филиалах в разных странных за некоторый период. Данные представлены в след. формате: `страна;компания;дата;доход`. Вам необходимо найти максимальный доход у каждой компании в отдельности по каждой представленной стране в данных.

Результат работы скрипта должен выглядеть следующим образом:

```bash
country  company                    income
Austria  Rodriguez, Sims and Arias  123456
USA      Rios-Lewis                 789998
Canada   Thompson-Mendez            148345
```

**Рекомендации**

1. В рамках этой задачи вам могу пригодиться дополнительные параметры mr таски, отвечающие за настройку шаффла, и правил сортировки ключей внутри него. Почитать о примерах их использования можно [здесь](https://hadoop.apache.org/docs/current/hadoop-streaming/HadoopStreaming.html#More_Usage_Examples).

2. Не рекомендуем использовать `\t` в качестве символа разделителя для сложного ключа (потому что по дефолту таб используется для разделения колонок данных, и ключом в таком случае будет только первая колонка до таба). Если вы будете собирать сложный ключ для нужной вам сортировки данных, лучше всего будет использовать другие симловы, к примеру `+, =`

3. Как и в задании 2.1, вы можете создать набор ячеек, куда занесете код своих мапперы и редьюсеры, и также соберете код запуска MR таски. Нам нужно, чтобы вы предоставили нам в качестве решения этой задачи все эти составляющие в рамках этого ноутбука.

In [54]:
%%writefile mapper2.py
import sys

for line in sys.stdin:
    line = line.strip()
    country, company, date, income = line.split(';')
    # Выводим ключ в формате "country+company" для корректной сортировки
    print(f"{country}+{company}\t{income}")

Writing mapper2.py


In [53]:
%%writefile reducer2.py
import sys

current_key = None
max_income = -1

for line in sys.stdin:
    line = line.strip()
    key, income = line.split('\t')
    income = int(income)
    
    if current_key == key:
        if income > max_income:
            max_income = income
    else:
        if current_key:
            country, company = current_key.split('+')
            print(f"{country}\t{company}\t{max_income}")
        current_key = key
        max_income = income

# Выводим последнюю пару ключ-значение
if current_key:
    country, company = current_key.split('+')
    print(f"{country}\t{company}\t{max_income}")

Writing reducer2.py


In [57]:
%%bash

# Очистка директории с результатом предыдущего выполнения
hdfs dfs -rm -r /output || true

# Запуск MapReduce задачи
hadoop jar /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -D mapreduce.job.name="max-income-per-company-per-country" \
    -files mapper2.py,reducer2.py \
    -mapper "python3 mapper2.py" \
    -reducer "python3 reducer2.py" \
    -input /input/company_revenue/income_country.csv \
    -output /output

Deleted /output
packageJobJar: [/tmp/hadoop-unjar1918019978632213960/] [] /tmp/streamjob1503766123096066745.jar tmpDir=null
2024-06-16 14:08:57 INFO  DefaultNoHARMFailoverProxyProvider:64 - Connecting to ResourceManager at resourcemanager/172.22.0.5:8032
2024-06-16 14:08:57 INFO  DefaultNoHARMFailoverProxyProvider:64 - Connecting to ResourceManager at resourcemanager/172.22.0.5:8032
2024-06-16 14:08:58 INFO  JobResourceUploader:907 - Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1718538847154_0003
2024-06-16 14:08:58 INFO  FileInputFormat:266 - Total input files to process : 1
2024-06-16 14:08:58 INFO  JobSubmitter:202 - number of splits:2
2024-06-16 14:08:58 INFO  JobSubmitter:298 - Submitting tokens for job: job_1718538847154_0003
2024-06-16 14:08:58 INFO  JobSubmitter:299 - Executing with tokens: []
2024-06-16 14:08:58 INFO  Configuration:2854 - resource-types.xml not found
2024-06-16 14:08:58 INFO  ResourceUtils:476 - Unable to find 'resource-types.x

In [58]:
%%bash
## запустите эту команду, чтобы вывести результат работы по определенным компаниям, которые мы указали на grep
## Это нам будет необходимо для визуального анализа результата работы вашего скрипта
## в sort можете указать тот разделитель колонок, с которым у вас результат выплевывает редьюсер
## укажите путь до той директории на hdfs, куда вы складывали результат

hdfs dfs -cat /output/* | \
grep -E "China.*?Rios-Lewis|India.*?Thompson-Mendez|Kazakhstan.*?Moore, Woods and Thomas|Madagascar.*?Chavez Group|New Zealand.*?Rodriguez, Sims and Arias" | \
column -t -s$'\t'

British Indian Ocean Territory (Chagos Archipelago)  Thompson-Mendez            974420
China                                                Rios-Lewis                 994876
India                                                Thompson-Mendez            953060
Kazakhstan                                           Moore, Woods and Thomas    994117
Madagascar                                           Chavez Group               986514
New Zealand                                          Rodriguez, Sims and Arias  888292


**2.3 Решаем задачу поиска самых посещаемых сайтов**

В данном задании нужно поработать с логом данных о посещении юзерами различных сайтов.
Формат данных: `url;временная метка`. Вам нужно вывести топ 5 сайтов по посещаемости в каждую из дат, которая представлена в наших данных.

Результат работы скрипта должен выглядеть следующим образом:

```bash
date        site                            count
2024-05-25  https://gonzales-bautista.com/  987
2024-05-25  https://smith.com/              654
2024-05-25  https://www.smith.com/          321
```

**Рекомендации**

1. Вам могу пригодиться дополнительные параметры mr таски, отвечающие за настройку шаффла, и правил сортировки ключей внутри него. Почитать о примерах их использования можно [здесь](https://hadoop.apache.org/docs/current/hadoop-streaming/HadoopStreaming.html#More_Usage_Examples).

2. Не рекомендуем использовать `\t` в качестве символа разделителя для сложного ключа (потому что по дефолту таб используется для разделения колонок данных, и ключом в таком случае будет только первая колонка до таба). Если вы будете собирать сложный ключ для нужной вам сортировки данных, лучше всего будет использовать другие симловы, к примеру `+, =`.

3. Возможно, у вас не получится решить данную задачу за одну mr таску, тогда вы просто описываете в решении скрипты ваших мапперов, редьюсеров под каждую из mr тасок, которые вам нужно запустить для получения нужного результата.

**Важно** помнить, что любой маппер и редьюсер должен работать за O(1) памяти, и если вы будете создавать какой-то список, куда будете складывать какие-то данные, то он не должен быть размера O(n). Если такой момент в вашем решении будет, пожалуйста, поясните его текстово, что с вашими переменными все ок, и у них нет размера O(n). 

In [59]:
%%writefile mapper3.py
import sys

for line in sys.stdin:
    line = line.strip()
    if not line:
        continue
    parts = line.split(';')
    if len(parts) != 2:
        continue
    url, timestamp = parts
    date = timestamp[:10]  
    key = f"{date}+{url}"
    value = 1
    print(f"{key}\t{value}")


Writing mapper3.py


In [60]:
%%writefile reducer3.py
import sys

current_date = None
current_top_sites = {}
MAX_SITES = 5

def emit_top_sites(date, top_sites):
    sorted_sites = sorted(top_sites.items(), key=lambda x: x[1], reverse=True)
    for i, (site, count) in enumerate(sorted_sites[:MAX_SITES]):
        print(f"{date}\t{site}\t{count}")

for line in sys.stdin:
    line = line.strip()
    if not line:
        continue
    key, value = line.split('\t', 1)
    date, url = key.split('+', 1)
    count = int(value)
    
    if current_date != date:
        if current_date is not None:
            emit_top_sites(current_date, current_top_sites)
        current_date = date
        current_top_sites = {}
    
    if url in current_top_sites:
        current_top_sites[url] += count
    else:
        current_top_sites[url] = count

# Emit top sites for the last date
if current_date is not None:
    emit_top_sites(current_date, current_top_sites)


Writing reducer3.py


In [61]:
%%bash

# Очистка директории с результатом предыдущего выполнения
hdfs dfs -rm -r /output || true

# Запуск MapReduce задачи
hadoop jar /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -D mapreduce.job.name="top-5-sites-per-date" \
    -files mapper3.py,reducer3.py \
    -mapper "python3 mapper3.py" \
    -reducer "python3 reducer3.py" \
    -input /input/website_logs/data_url.csv \
    -output /output


Deleted /output
packageJobJar: [/tmp/hadoop-unjar6996084681257687832/] [] /tmp/streamjob6907223935893469304.jar tmpDir=null
2024-06-16 14:37:47 INFO  DefaultNoHARMFailoverProxyProvider:64 - Connecting to ResourceManager at resourcemanager/172.22.0.5:8032
2024-06-16 14:37:47 INFO  DefaultNoHARMFailoverProxyProvider:64 - Connecting to ResourceManager at resourcemanager/172.22.0.5:8032
2024-06-16 14:37:47 INFO  JobResourceUploader:907 - Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1718538847154_0004
2024-06-16 14:37:47 INFO  FileInputFormat:266 - Total input files to process : 1
2024-06-16 14:37:47 INFO  JobSubmitter:202 - number of splits:2
2024-06-16 14:37:47 INFO  JobSubmitter:298 - Submitting tokens for job: job_1718538847154_0004
2024-06-16 14:37:47 INFO  JobSubmitter:299 - Executing with tokens: []
2024-06-16 14:37:47 INFO  Configuration:2854 - resource-types.xml not found
2024-06-16 14:37:47 INFO  ResourceUtils:476 - Unable to find 'resource-types.x

In [63]:
%%bash
## запустите эту команду, чтобы вывести результат работы по определенным компаниям, которые мы указали на grep
## Это нам будет необходимо для визуального анализа результата работы вашего скрипта
## в sort можете указать тот разделитель колонок, с которым у вас результат выплевывает редьюсер
## укажите путь до той директории на hdfs, куда вы складывали результат

hdfs dfs -cat /output/* | grep -E '2024-05-28|2024-06-02|2024-05-30' | column -t -s$'\t'

2024-05-28  https://gonzales-bautista.com/  368
2024-05-28  https://smith.com/              256
2024-05-28  https://www.smith.com/          251
2024-05-28  http://smith.com/               224
2024-05-28  http://www.smith.com/           204
2024-05-30  https://gonzales-bautista.com/  353
2024-05-30  https://smith.com/              246
2024-05-30  https://www.smith.com/          239
2024-05-30  http://smith.com/               229
2024-05-30  http://www.smith.com/           225
2024-06-02  http://smith.com/               7
2024-06-02  https://gonzales-bautista.com/  7
2024-06-02  https://www.williams.com/       6
2024-06-02  http://lee.com/                 5
2024-06-02  http://miller.com/              5


In [70]:
!git init

/usr/bin/sh: git: command not found


In [69]:
%%bash

!git init
!git remote add origin https://github.com/pvlnkmaria/hadoop-map-reduce.git
!git add HW_1_Bash_Hadoop_MapReduce.ipynb  
!git commit -m "first commit"
!git push -u origin main  


bash: line 2: !git: command not found
bash: line 3: !git: command not found
bash: line 4: !git: command not found
bash: line 5: !git: command not found
bash: line 6: !git: command not found


CalledProcessError: Command 'b'\n!git init\n!git remote add origin https://github.com/pvlnkmaria/hadoop-map-reduce.git\n!git add HW_1_Bash_Hadoop_MapReduce.ipynb  \n!git commit -m "first commit"\n!git push -u origin main  \n'' returned non-zero exit status 127.